In [1]:
# --- Standard Library ---
import os
import sys
import json

# --- Google Cloud Auth + APIs ---
from google.cloud import bigquery
from google.oauth2 import service_account
from google.api_core.exceptions import GoogleAPICallError, RetryError

import gspread
from gspread_dataframe import set_with_dataframe

# --- Data & Visualization ---
import pandas as pd
import pandas_gbq
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

# --- Local Modules ---
from modules.pull_and_backup import pull_and_append
from modules.flattening import flatten_extract_params, flatten_row, flatten_nested_column
from modules.column_order import column_order

In [2]:
# --- Configuration ---
SERVICE_ACCOUNT_KEY = './keys/key.json'
DATA_PATH = './data/data.json'
PROJECT_ID = "emojioracle-342f1"
DATASET_ID = "analytics_481352676"
BACKUP_PATH = './backup/'
SCOPES = [
    "https://www.googleapis.com/auth/bigquery",
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]
# --- BigQuery Client ---
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_KEY,
    scopes = SCOPES
)
bq_client = bigquery.Client(credentials = credentials, project = PROJECT_ID)

In [3]:
# will pull what is missing from BigQuery and merge into data_path
df = pd.DataFrame(pull_and_append(credentials = credentials, project_id = PROJECT_ID, dataset_id = DATASET_ID, data_path = DATA_PATH, backup_path = BACKUP_PATH))

Loaded existing data.
Latest event_date in merged data: 20250522
Backup already exists: events_20250405
Backup already exists: events_20250406
Backup already exists: events_20250407
Backup already exists: events_20250408
Backup already exists: events_20250409
Backup already exists: events_20250410
Backup already exists: events_20250411
Backup already exists: events_20250412
Backup already exists: events_20250413
Backup already exists: events_20250414
Backup already exists: events_20250415
Backup already exists: events_20250416
Backup already exists: events_20250417
Backup already exists: events_20250418
Backup already exists: events_20250420
Backup already exists: events_20250421
Backup already exists: events_20250422
Backup already exists: events_20250423
Backup already exists: events_20250424
Backup already exists: events_20250425
Backup already exists: events_20250427
Backup already exists: events_20250428
Backup already exists: events_20250429
Backup already exists: events_20250430

In [4]:
# load the data from the JSON file
df = pd.read_json(DATA_PATH)

In [5]:
df.tail()

,event_date,event_timestamp,event_name,event_params,event_previous_timestamp,event_value_in_usd,event_bundle_sequence_id,event_server_timestamp_offset,user_id,user_pseudo_id,...,event_dimensions,ecommerce,items,collected_traffic_source,is_active_user,batch_event_index,batch_page_id,batch_ordering_id,session_traffic_source_last_click,publisher
19321,20250528,1748462561445007,user_engagement,"[{'key': 'firebase_screen_class', 'value': {'s...",1.748462e+15,NaN,52,1811168,NaN,527d79103a664d002beabedcfdaa5b14,...,NaN,NaN,[],NaN,True,8,NaN,NaN,{'manual_campaign': {'campaign_id': '(not set)...,NaN
19322,20250528,1748463390213000,session_start,"[{'key': 'ga_session_number', 'value': {'strin...",NaN,NaN,3,806,NaN,a79d23b40c843fcef55e9a9013b90a35,...,NaN,NaN,[],NaN,True,1,NaN,NaN,{'manual_campaign': {'campaign_id': '(not set)...,NaN
19323,20250528,1748463390266001,screen_view,"[{'key': 'ga_session_id', 'value': {'string_va...",NaN,NaN,3,806,NaN,a79d23b40c843fcef55e9a9013b90a35,...,NaN,NaN,[],NaN,True,2,NaN,NaN,{'manual_campaign': {'campaign_id': '(not set)...,NaN
19324,20250528,1748463554297000,question_completed,"[{'key': 'engaged_session_event', 'value': {'s...",NaN,NaN,7,637,NaN,a79d23b40c843fcef55e9a9013b90a35,...,NaN,NaN,[],NaN,True,1,NaN,NaN,{'manual_campaign': {'campaign_id': '(not set)...,NaN
19325,20250528,1748463558675001,earn_virtual_currency,"[{'key': 'engaged_session_event', 'value': {'s...",NaN,NaN,7,637,NaN,a79d23b40c843fcef55e9a9013b90a35,...,NaN,NaN,[],NaN,True,2,NaN,NaN,{'manual_campaign': {'campaign_id': '(not set)...,NaN


In [6]:
pd.set_option('display.max_columns', None) # uncomment to see all of the cols in pandas dataframes

In [7]:
# flattenning

df = pd.DataFrame([flatten_row(row) for _, row in df.iterrows()]) # for wtfs refer to ./modules/flattening_json.py

In [8]:
# cleaning & preprocessing

## dates and times
df = df.drop(columns=['event_date']) # built in case event_date may not be the same as the one in the event_timestamp

# convert unix time to ape-friendly datetime format
df['event_datetime'] = pd.to_datetime(df['event_timestamp'], unit='us', utc=True) 
df['event_previous_datetime'] = pd.to_datetime(df['event_previous_timestamp'], unit='us', utc=True)
df['event_first_touch_datetime'] = pd.to_datetime(df['user_first_touch_timestamp'], unit='us', utc=True)
df['user.first_open_datetime'] = pd.to_datetime(df['user.first_open_time'], unit='ms', utc=True)


df['event_date'] = df['event_datetime'].dt.normalize()
df['event_time'] = df['event_datetime'].dt.time

df['event_previous_date'] = df['event_previous_datetime'].dt.date
df['event_previous_time'] = df['event_previous_datetime'].dt.time

df['event_first_touch_date'] = df['event_first_touch_datetime'].dt.date
df['event_first_touch_time'] = df['event_first_touch_datetime'].dt.time

df['user.first_open_date'] = df['user.first_open_datetime'].dt.date
df['user.first_open_time'] = df['user.first_open_datetime'].dt.time

df['device.time_zone_offset_hours'] = df['device.time_zone_offset_seconds'] / 3600 # seconds to hours
df['event_params.engagement_time_seconds'] = df['event_params.engagement_time_msec'] / 1000 # ms to seconds
df['event_server_delay_seconds'] = df['event_server_timestamp_offset'] / 1000 # ms to seconds 
df['event_params.time_spent_seconds'] = df['event_params.time_spent'] # just renaming for clarity

df = df.drop(columns=['event_timestamp',
                      'event_previous_timestamp', 
                      'user_first_touch_timestamp', 
                      'event_server_timestamp_offset', 
                      'device.time_zone_offset_seconds', 
                      'event_params.engagement_time_msec',
                      'event_previous_datetime',
                      'event_params.time_spent',
                      'event_first_touch_datetime',
                      'user.first_open_datetime'
                    ])



In [9]:
## time series

df['ts_weekday'] = df['event_datetime'].dt.day_name() # weekday name
df['ts_weekday'] = pd.Categorical(df['ts_weekday'], categories=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], ordered=True) # order the weekdays

df['ts_local_time'] = df['event_datetime'] + pd.to_timedelta(df['device.time_zone_offset_hours'].fillna(0), unit='h') # local time
df['ts_hour'] = df['ts_local_time'].dt.hour # local hour
df['ts_daytime_named'] = df['ts_hour'].apply(lambda x: 
                                             'night' if (x < 6 or x > 22) else 
                                             'morning' if x < 11 else 
                                             'noon' if x < 14 else 
                                             'afternoon' if x < 17 else 
                                             'evening') # time group of day
df['ts_is_weekend'] = df['ts_weekday'].isin(['Saturday', 'Sunday']) # is weekend

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 99 columns):
 #   Column                                  Non-Null Count  Dtype              
---  ------                                  --------------  -----              
 0   event_name                              19326 non-null  object             
 1   event_value_in_usd                      0 non-null      float64            
 2   event_bundle_sequence_id                19326 non-null  int64              
 3   user_id                                 0 non-null      float64            
 4   user_pseudo_id                          19326 non-null  object             
 5   stream_id                               19326 non-null  int64              
 6   platform                                19326 non-null  object             
 7   is_active_user                          19326 non-null  bool               
 8   batch_event_index                       19326 non-null  int64              


In [11]:
# Revise Question Indices as df['event_params.current_question_index'] 
"""
Tier 1: 16 Questions, Except t: 12
Tier 2: 12 Questions
Tier 3: 12 Questions
Tier 4: 10 Questions
"""

df['event_params.current_question_index'] = pd.Series([pd.NA] * len(df), dtype="Int64")

df['event_params.current_tier'] = pd.to_numeric(df['event_params.current_tier'], errors='coerce').astype("Int64")
df['event_params.current_qi'] = pd.to_numeric(df['event_params.current_qi'], errors='coerce').astype("Int64")

notna_mask = df['event_params.character_name'].notna()

# Tier 1
tier_1_mask = notna_mask & (df['event_params.current_tier'] == 1)
t_char_mask = tier_1_mask & (df['event_params.character_name'] == 't')

df.loc[t_char_mask, 'event_params.current_question_index'] = 13 - df.loc[t_char_mask, 'event_params.current_qi']
df.loc[~t_char_mask & tier_1_mask, 'event_params.current_question_index'] = 17 - df.loc[(~t_char_mask) & tier_1_mask, 'event_params.current_qi']

# Tier 2 & 3
tier_2_3_mask = notna_mask & df['event_params.current_tier'].isin([2, 3])
df.loc[tier_2_3_mask, 'event_params.current_question_index'] = 13 - df.loc[tier_2_3_mask, 'event_params.current_qi']

# Tier 4
tier_4_mask = notna_mask & (df['event_params.current_tier'] == 4)
df.loc[tier_4_mask, 'event_params.current_question_index'] = 7 - df.loc[tier_4_mask, 'event_params.current_qi']

# Hiccups
problems_mask = notna_mask & ~df['event_params.current_tier'].isin([1, 2, 3, 4])
if df[problems_mask].shape[0] > 0:
    print("Something wrong in:")
    print(df.loc[problems_mask, ['event_params.character_name', 'event_params.current_tier', 'event_params.current_qi']])


In [12]:
# I reordered the columns to make it easier to follow, in column_order.py

df = df[column_order]

df.columns = df.columns.str.replace('.', '__') # replace dots with dashes in column names bc of bigquerys nagging

In [13]:
df.columns

Index(['event_name', 'event_date', 'event_time', 'event_previous_date',
       'event_previous_time', 'event_first_touch_date',
       'event_first_touch_time', 'event_bundle_sequence_id', 'user_id',
       'user_pseudo_id', 'user__first_open_date', 'user__first_open_time',
       'user__ga_session_id', 'user__ga_session_number', 'app_info__id',
       'app_info__firebase_app_id', 'app_info__version',
       'app_info__install_store', 'app_info__install_source',
       'device__advertising_id', 'device__vendor_id', 'device__category',
       'device__mobile_brand_name', 'device__mobile_model_name',
       'device__mobile_marketing_name', 'device__mobile_os_hardware_model',
       'device__operating_system', 'device__operating_system_version',
       'device__language', 'device__is_limited_ad_tracking', 'device__browser',
       'device__browser_version', 'device__web_info',
       'device__time_zone_offset_hours', 'geo__city', 'geo__country',
       'geo__continent', 'geo__region', 'ge

In [14]:
df.tail(5)

,event_name,event_date,event_time,event_previous_date,event_previous_time,event_first_touch_date,event_first_touch_time,event_bundle_sequence_id,user_id,user_pseudo_id,user__first_open_date,user__first_open_time,user__ga_session_id,user__ga_session_number,app_info__id,app_info__firebase_app_id,app_info__version,app_info__install_store,app_info__install_source,device__advertising_id,device__vendor_id,device__category,device__mobile_brand_name,device__mobile_model_name,device__mobile_marketing_name,device__mobile_os_hardware_model,device__operating_system,device__operating_system_version,device__language,device__is_limited_ad_tracking,device__browser,device__browser_version,device__web_info,device__time_zone_offset_hours,geo__city,geo__country,geo__continent,geo__region,geo__sub_continent,geo__metro,traffic_source__name,traffic_source__medium,traffic_source__source,collected_traffic_source,event_params__ga_session_id,event_params__firebase_screen_id,event_params__ad_unit_id,event_params__ad_format,event_params__ad_network,event_params__ad_platform,event_params__ad_shown_where,event_params__answered_wrong,event_params__character_name,event_params__current_qi,event_params__current_question_index,event_params__current_tier,event_params__earned_amount,event_params__engaged_session_event,event_params__engagement_time_seconds,event_params__entrances,event_params__firebase_conversion,event_params__firebase_error,event_params__firebase_event_origin,event_params__firebase_screen_class,event_params__ga_session_number,event_params__how_its_earned,event_params__menu_name,event_params__mini_game_name,event_params__mini_game_ri,event_params__previous_first_open_count,event_params__session_engaged,event_params__spent_amount,event_params__spent_to,event_params__system_app,event_params__system_app_update,event_params__time_spent_seconds,event_params__update_with_analytics,event_params__where_its_earned,event_params__where_its_spent,event_params__currency_name,batch_event_index,batch_ordering_id,batch_page_id,privacy_info__ads_storage,privacy_info__analytics_storage,privacy_info__uses_transient_token,event_dimensions,event_server_delay_seconds,event_value_in_usd,ecommerce,is_active_user,platform,stream_id,user_ltv,event_datetime,ts_weekday,ts_is_weekend,ts_local_time,ts_hour,ts_daytime_named
19321,user_engagement,2025-05-28 00:00:00+00:00,20:02:41.445007,2025-05-28,19:59:57.411007,2025-05-28,19:52:40.397000,52,NaN,527d79103a664d002beabedcfdaa5b14,2025-05-28,20:00:00,1.748462e+09,1.0,com.TestCompany.TestApp,1:332513280181:android:e9c9cb25732e9477377efc,0.1.0,None,com.google.android.packageinstaller,None,None,tablet,Aidata,ADT1061,None,ADT1061,Android,Android 10,tr-tr,No,None,None,None,3.0,Gaziantep,Türkiye,Asia,Gaziantep,Western Asia,(not set),(direct),(none),(direct),None,1748461961,-8.445675e+18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,1.0,159.449,NaN,NaN,NaN,auto,UnityPlayerActivity,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,NaN,NaN,Yes,Yes,No,None,1811.168,NaN,None,True,ANDROID,10359646141,None,2025-05-28 20:02:41.445007+00:00,Wednesday,False,2025-05-28 23:02:41.445007+00:00,23,night
19322,session_start,2025-05-28 00:00:00+00:00,20:16:30.213000,NaT,NaT,2025-05-28,20:16:29.827000,3,NaN,a79d23b40c843fcef55e9a9013b90a35,2025-05-28,21:00:00,1.748463e+09,1.0,com.TestCompany.TestApp,1:332513280181:android:e9c9cb25732e9477377efc,0.1.0,None,com.google.android.packageinstaller,None,None,mobile,Xiaomi,220333QAG,Redmi 10C,220333QAG,Android,Android 13,tr-tr,No,None,None,None,3.0,Ankara,Türkiye,Asia,Ankara,Western Asia,(not set),(direct),(none),(direct),None,1748463390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,1.0,NaN,NaN,NaN,NaN,auto,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,Yes,Yes,No,None,0.806,NaN,None,True,ANDROID,10359646141,None,2025-05-28 20:16:30.213000+00:00,Wednesday,False,2025-05-28 23:16:30.213000+00:00,23,night
19323,screen_view,2025-05-28 00:00:00+00:00,20:16:30.266001

In [15]:
df.to_parquet('./data/cleaned_data.parquet', index=False)

In [16]:
# Write to BigQuery 
table_id = f"{PROJECT_ID}.{DATASET_ID}.clean_data"

try:
    # Load data into BigQuery
    job = bq_client.load_table_from_dataframe(df, table_id, job_config = bigquery.LoadJobConfig(
        write_disposition = "WRITE_TRUNCATE"
    ))

    job.result()  # Wait for the job to finish

    print(f"Cleaned data written to {table_id}. Ready to use in Looker.")

except GoogleAPICallError as api_error:
    print(f"API error occurred: {api_error}")
except RetryError as retry_error:
    print(f"Retry error occurred: {retry_error}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Cleaned data written to emojioracle-342f1.analytics_481352676.clean_data. Ready to use in Looker.


In [17]:
df['event_params__mini_game_name'].unique()

array([nan, 'stone_mini_game', 'cauldron_mini_game', 'star_mini_game',
       'maze_mini_game', 'coffee_mini_game', 'card_mini_game',
       'wheel_of_fortune', 'voodoo_mini_game'], dtype=object)